In [ ]:
!pip install psycopg2-binary nltk unidecode gensim

## PGVECTOR
Extensão para o PostreSQL que permite manipular vetores de alta dimensionalidade diretamente no banco de dados.
Principais caso de uso:
- Recuperação de informação (busca por similaridade entre vetores. Sistema de recomendação, pesquisa semântica, classificação de documentos);
- Análise de texto (análise de sentimento, agrupamento de documentos, detecção de spam);
- Processamento de linguagem natural (PLN/NLP);
- Análise de imagens  (buscar características de imagens em formato vetorial)

## Embeddings
Representação numérica de um objeto (palavra, frase, imagem, entidade) em espaço vetorial. Embeddings são projetados de forma que objetos semelhantes estejam próximos uns dos outros.

## Tokenização
Processo que envolve dividir um texto em unidades menores (tokens) que podem ser palavras, subpalavras ou caracteres.

## Word2Vec
Técnica de aprendizado de representação de palavras que mapeia palavras do vocavulário para vetores numéricos, permitindo a captura de semelhanças semânticas e relações entre palavras.

## Dockerfile
```
FROM postgres:latest

RUN apt-get update && apt-get install -y postgresql-16-pgvector
```

## docker-compose.yml
```
version: '3.8'

services:
  db:
    build:
      context: .
      dockerfile: Dockerfile
    container_name: pgvector-container
    ports:
      - "5432:5432"
    environment:
      POSTGRES_PASSWORD: password
      POSTGRES_USER: postgres
      POSTGRES_DB: mydatabase
```

In [5]:
frases = [
    "João gostava de estudar Ciência de Dados.",
    "Pedro gosta de comer chocolate.",
    "O cãozinho de Maria é muito bagunceiro.",
    "Jonas detesta chocolate amargo.",
    "Eles voltaram muito tarde da festa de ontem.",
    "As ondas do mar estavam agitadas no último final de semana.",
    "O pôr do sol estava magnífico naquele dia.",
    "Amanhã será um dia ensolarado.",
    "O gato de João fugiu de casa na semana passada.",
    "Joaquina gosta de João.",
    "Pedro detesta Maria.",
    "Maria estuda no colégio mais caro da cidade.",
    "Maria, Pedro e João estudam na mesma escola."
]

In [6]:
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
nltk.download('rslp') # conjunto de dados com modelos pré-treinados em pt

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [7]:
tokenized_sentences = [word_tokenize(frase) for frase in frases]
tokenized_sentences

[['João', 'gostava', 'de', 'estudar', 'Ciência', 'de', 'Dados', '.'],
 ['Pedro', 'gosta', 'de', 'comer', 'chocolate', '.'],
 ['O', 'cãozinho', 'de', 'Maria', 'é', 'muito', 'bagunceiro', '.'],
 ['Jonas', 'detesta', 'chocolate', 'amargo', '.'],
 ['Eles', 'voltaram', 'muito', 'tarde', 'da', 'festa', 'de', 'ontem', '.'],
 ['As',
  'ondas',
  'do',
  'mar',
  'estavam',
  'agitadas',
  'no',
  'último',
  'final',
  'de',
  'semana',
  '.'],
 ['O', 'pôr', 'do', 'sol', 'estava', 'magnífico', 'naquele', 'dia', '.'],
 ['Amanhã', 'será', 'um', 'dia', 'ensolarado', '.'],
 ['O',
  'gato',
  'de',
  'João',
  'fugiu',
  'de',
  'casa',
  'na',
  'semana',
  'passada',
  '.'],
 ['Joaquina', 'gosta', 'de', 'João', '.'],
 ['Pedro', 'detesta', 'Maria', '.'],
 ['Maria', 'estuda', 'no', 'colégio', 'mais', 'caro', 'da', 'cidade', '.'],
 ['Maria', ',', 'Pedro', 'e', 'João', 'estudam', 'na', 'mesma', 'escola', '.']]

In [8]:
modelo = Word2Vec(tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

In [9]:
vetores = {}
for palavra in modelo.wv.key_to_index:
    vetores[palavra] = modelo.wv[palavra]

In [10]:
vetores.keys()

dict_keys(['.', 'de', 'João', 'Maria', 'Pedro', 'O', 'na', 'dia', 'muito', 'do', 'semana', 'chocolate', 'da', 'gosta', 'no', 'detesta', 'Eles', 'festa', 'ontem', 'tarde', 'voltaram', 'As', 'é', 'amargo', 'Jonas', 'bagunceiro', 'cãozinho', 'comer', 'Dados', 'Ciência', 'estudar', 'gostava', 'ondas', 'escola', 'mar', 'mesma', 'estudam', 'e', ',', 'cidade', 'caro', 'mais', 'colégio', 'estuda', 'Joaquina', 'passada', 'casa', 'fugiu', 'gato', 'ensolarado', 'um', 'será', 'Amanhã', 'naquele', 'magnífico', 'estava', 'sol', 'pôr', 'final', 'último', 'agitadas', 'estavam'])

## Processar o texto?
Em alguns casos pode ser interessante processar o texto para facilitar o alcance do objetivo do modelo que será treinado. Essa processamento pode incluir a remoção de caracteres especiais (como acentos e pontuação), a retirada de algumas palavras que podem ser pouco interessante para a propsota do modelo e até mesmo a remoção de prefixos e sufixos de palavras, deixando apenas o radical da palavra. Ainda existem casos de remoção de palavras inteiras (stop words).

### Vantagens de processar o texto
- Reduz a variabilidade;
- Simplifica o vocabulário;
- Melhora a qualidade dos vetores;
- Facilita a comparação de palavras.


### Quando processar?
- Análise de sentimentos;
- Classificação de texto.

### Quando não processar?
- Análise de texto multilíngue;
- Preservação de informações contextuais.

### Stop Words
Palavras comuns geralmente são removidas por serem consideradas menos imortantes para a compreensão do significado do texto. É interessante removê-las ao treinar um modelo Word2Vect quando elas não contribuem significativamente para a representação semântica das frases e podem introduzir ruídos  nos vetores. Removê-las ajuda a melhorar a qualidade dos vetores gerados pelo modelo, dando foco nas palavras mais informativas.




In [11]:
import string
from unidecode import unidecode

tabela_pontuacao = str.maketrans('', '', string.punctuation) # tabela de tradução para remover pontuação
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
frases_processadas = []
for frase in frases:
    nova_frase = unidecode(frase.lower())
    nova_frase = nova_frase.translate(tabela_pontuacao)
    frases_processadas.append(nova_frase)
frases_processadas

['joao gostava de estudar ciencia de dados',
 'pedro gosta de comer chocolate',
 'o caozinho de maria e muito bagunceiro',
 'jonas detesta chocolate amargo',
 'eles voltaram muito tarde da festa de ontem',
 'as ondas do mar estavam agitadas no ultimo final de semana',
 'o por do sol estava magnifico naquele dia',
 'amanha sera um dia ensolarado',
 'o gato de joao fugiu de casa na semana passada',
 'joaquina gosta de joao',
 'pedro detesta maria',
 'maria estuda no colegio mais caro da cidade',
 'maria pedro e joao estudam na mesma escola']

In [13]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))
stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'a',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estar',
 'estas',
 'estava',
 'estavam',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fui',
 'fôramos',
 'fôssemos',
 'haja',
 'hajam',
 'hajamos',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houveram',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'houveriam',
 'houvermos',
 'houverá',
 'houverão',
 'houverí

In [14]:
frases_sem_stopwords = []
for frase in frases_processadas:
    tokens = word_tokenize(frase)
    tokens_sem_stopwords = [token for token in tokens if token not in stop_words]
    frases_sem_stopwords.append(tokens_sem_stopwords)
frases_sem_stopwords

[['joao', 'gostava', 'estudar', 'ciencia', 'dados'],
 ['pedro', 'gosta', 'comer', 'chocolate'],
 ['caozinho', 'maria', 'bagunceiro'],
 ['jonas', 'detesta', 'chocolate', 'amargo'],
 ['voltaram', 'tarde', 'festa', 'ontem'],
 ['ondas', 'mar', 'agitadas', 'ultimo', 'final', 'semana'],
 ['sol', 'magnifico', 'naquele', 'dia'],
 ['amanha', 'sera', 'dia', 'ensolarado'],
 ['gato', 'joao', 'fugiu', 'casa', 'semana', 'passada'],
 ['joaquina', 'gosta', 'joao'],
 ['pedro', 'detesta', 'maria'],
 ['maria', 'estuda', 'colegio', 'caro', 'cidade'],
 ['maria', 'pedro', 'joao', 'estudam', 'mesma', 'escola']]

In [15]:
model_no_stopwords = Word2Vec(frases_sem_stopwords, vector_size=100, window=5, min_count=1, workers=4)

In [16]:
vetores_pgvector = {}
for palavra in model_no_stopwords.wv.key_to_index:
    vetores_pgvector[palavra] = model_no_stopwords.wv[palavra]

In [17]:
vetores_pgvector['joao']

array([-5.4647954e-04,  2.2584017e-04,  5.1015122e-03,  9.0103038e-03,
       -9.3133766e-03, -7.1234256e-03,  6.4616804e-03,  8.9799482e-03,
       -5.0202948e-03, -3.7688590e-03,  7.3812045e-03, -1.5383899e-03,
       -4.5441505e-03,  6.5576700e-03, -4.8700930e-03, -1.8123193e-03,
        2.8764834e-03,  9.9321571e-04, -8.2821585e-03, -9.4561912e-03,
        7.3133963e-03,  5.0778156e-03,  6.7643747e-03,  7.6566049e-04,
        6.3580386e-03, -3.4075845e-03, -9.4601139e-04,  5.7735941e-03,
       -7.5312802e-03, -3.9393934e-03, -7.5085098e-03, -9.2570693e-04,
        9.5463926e-03, -7.3265536e-03, -2.3370429e-03, -1.9368682e-03,
        8.0819856e-03, -5.9340512e-03,  4.5197543e-05, -4.7506811e-03,
       -9.5978351e-03,  5.0087199e-03, -8.7746382e-03, -4.3968926e-03,
       -2.8846580e-05, -2.9816179e-04, -7.6701660e-03,  9.6239913e-03,
        4.9895034e-03,  9.2347767e-03, -8.1596468e-03,  4.5028282e-03,
       -4.1350583e-03,  8.2927366e-04,  8.5041607e-03, -4.4665299e-03,
      

In [30]:
import psycopg2
try:
    conn = psycopg2.connect(
        dbname="mydatabase",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )
    print("Conexão ao banco de dados PostgreSQL bem-sucedida!")
except psycopg2.Error as e:
    print("Erro ao conectar ao banco de dados PostgreSQL:", e)

cursor = conn.cursor()

Conexão ao banco de dados PostgreSQL bem-sucedida!


In [19]:
cursor.execute("""
                CREATE EXTENSION IF NOT EXISTS vector
               """)
conn.commit()

In [20]:
cursor.execute("""
    DROP TABLE IF EXISTS palavras;
               
    CREATE TABLE IF NOT EXISTS palavras (
        palavra VARCHAR PRIMARY KEY,
        vetor VECTOR
    );     
""")
conn.commit()

In [21]:
for palavra, vetor in vetores_pgvector.items():
    vetor_lista = vetor.tolist()
    cursor.execute("""
                    INSERT INTO palavras (palavra, vetor) VALUES (%s, %s)
                   """, (palavra, vetor_lista))

conn.commit()

In [22]:
cursor.execute("SELECT * FROM palavras WHERE palavra = 'joao'")
print(cursor.fetchone())

('joao', '[-0.00054647954,0.00022584017,0.005101512,0.009010304,-0.009313377,-0.0071234256,0.0064616804,0.008979948,-0.005020295,-0.003768859,0.0073812045,-0.00153839,-0.0045441505,0.00655767,-0.004870093,-0.0018123193,0.0028764834,0.0009932157,-0.0082821585,-0.009456191,0.0073133963,0.0050778156,0.0067643747,0.0007656605,0.0063580386,-0.0034075845,-0.0009460114,0.005773594,-0.00753128,-0.0039393934,-0.00750851,-0.00092570693,0.009546393,-0.0073265536,-0.002337043,-0.0019368682,0.008081986,-0.0059340512,4.5197543e-05,-0.004750681,-0.009597835,0.00500872,-0.008774638,-0.0043968926,-2.884658e-05,-0.0002981618,-0.007670166,0.009623991,0.0049895034,0.009234777,-0.008159647,0.0045028282,-0.0041350583,0.00082927366,0.008504161,-0.00446653,0.0045185336,-0.00679846,-0.0035486983,0.009400069,-0.0015764814,0.00032357415,-0.004134818,-0.007686556,-0.0015106944,0.0024689082,-0.00089483947,0.0055325376,-0.0027427245,0.0022547625,0.005461776,0.008350201,-0.001451077,-0.009215334,0.0043727043,0.00057

In [23]:
cursor.execute("""
    DROP TABLE IF EXISTS frases;
    
    CREATE TABLE IF NOT EXISTS frases (
        id INT PRIMARY KEY,
        frase_original TEXT,
        frase_processada TEXT,
        vetor_medio VECTOR
    )               
""")
conn.commit()

In [24]:
import numpy as np
def calcular_vetor_medio(vetores):
    if not vetores:
        return None
    return np.mean(vetores, axis=0)

In [25]:
vetores_frases = []
for frase_sem_stopwords in frases_sem_stopwords:
    vetores = [vetores_pgvector[palavra] for palavra in frase_sem_stopwords if palavra in vetores_pgvector]
    vetor_medio = calcular_vetor_medio(vetores)
    vetores_frases.append(vetor_medio)


In [27]:
for i, vetor in enumerate(vetores_frases):
    cursor.execute("""
                   INSERT INTO frases (id, frase_original, frase_processada, vetor_medio)
                   VALUES (%s, %s, %s, %s)
                   """,
                   (i+1, frases[i], ' '.join(frases_sem_stopwords[i]), vetor.tolist())
                   )
conn.commit()

UniqueViolation: duplicate key value violates unique constraint "frases_pkey"
DETAIL:  Key (id)=(1) already exists.


In [31]:
cursor.execute("SELECT * FROM frases WHERE id = 1")
print(cursor.fetchone())

(1, 'João gostava de estudar Ciência de Dados.', 'joao gostava estudar ciencia dados', '[-0.0009843915,0.0013499345,0.0042601265,0.007225538,-0.00032018227,0.0016333204,0.0012480567,0.0024502166,-0.0006330492,0.0012726279,0.0028621575,-0.0023449624,0.0024246182,0.0031382677,-0.0015405394,-0.0007161265,-0.0022034966,-0.00043933978,-0.005264936,-0.0034959875,0.0028117823,0.003761574,0.0016754474,0.0018044317,0.0021358952,0.0018088408,0.0004844314,-0.0024331056,-0.0022926782,-0.0031391673,0.0013739725,-0.0049681445,0.0046518464,-0.0021005461,0.00010945108,0.00010853154,0.002217096,-0.00016748719,-0.00092074525,-0.005265587,-0.00277698,7.727938e-05,-0.0043182364,0.0014041269,-0.0008110654,-0.003093484,-0.004139456,3.9967803e-05,0.0006629106,0.0027177634,-0.00014669579,-0.00068213156,-0.0025521684,-2.9078034e-05,0.00073985336,-0.001147366,0.0010252175,-0.0038997785,-0.00396998,-0.0017264324,-0.0013694796,0.0030565336,0.0011366396,-0.0009721086,-0.004217553,0.0028539593,-0.0014628444,0.00405

In [32]:
def processar_frase(frase):
    tabela_pontuacao = str.maketrans('', '', string.punctuation)
    frase = frase.translate(tabela_pontuacao)
    frase = unidecode(frase.lower())
    stop_words = set(stopwords.words('portuguese'))
    tokens = word_tokenize(frase.lower())
    return [token for token in tokens if token not in stop_words]

In [33]:
frase_teste = 'Roberto não gosta de chocolate.'
frase_teste_tokens = processar_frase(frase_teste)

vetores_teste = []
for palavra in frase_teste_tokens:
    print(palavra)
    cursor.execute("SELECT vetor FROM palavras WHERE palavra = %s", (palavra,))
    vetor = cursor.fetchone()
    if vetor:
        vetores_teste.append(np.array([float(v) for v in vetor[0].replace('[', '').replace(']', '').split(',')]))

roberto
nao
gosta
chocolate


In [34]:
vetor_medio_teste = calcular_vetor_medio([vetor_convertido.astype(np.float64) for vetor_convertido in vetores_teste])

In [35]:
vetor_medio_teste

array([ 0.00049554, -0.0016098 , -0.00412288, -0.00062056,  0.00176351,
        0.00349005,  0.00365598,  0.00104184, -0.00367784,  0.00285731,
       -0.00020399,  0.00308042, -0.00447335,  0.00568538, -0.00495076,
       -0.00254299,  0.00303313,  0.00620306,  0.00365064, -0.00692893,
        0.00096124, -0.00539215,  0.00859062,  0.00523334, -0.0006254 ,
        0.00160205, -0.00054334,  0.00023905, -0.00418783, -0.0007158 ,
        0.00673559,  0.00558637,  0.00022624, -0.00317325,  0.00118783,
       -0.00228381, -0.00495899, -0.00260047, -0.00333571, -0.00377296,
        0.00452925, -0.00630017, -0.00072551,  0.00089764,  0.0048105 ,
        0.00138103, -0.00219191, -0.0048679 ,  0.00021265, -0.00490126,
        0.00187753, -0.00224078,  0.0006703 , -0.00083671, -0.00479682,
       -0.00819418, -0.00601812, -0.00885705, -0.00404016, -0.00745154,
        0.0022743 , -0.00536371,  0.00457326,  0.00113259, -0.00336765,
        0.00256435,  0.00198295,  0.00224493, -0.00583357,  0.00

### Distância Euclidiana '<->'

In [36]:
cursor.execute('SELECT (vetor_medio <-> %s) AS distancia, frase_original FROM frases ORDER BY distancia;', (str(vetor_medio_teste.tolist()),))
cursor.fetchall()

[(0.027310343541644203, 'Pedro gosta de comer chocolate.'),
 (0.03802360422236187, 'Jonas detesta chocolate amargo.'),
 (0.04023367119608727, 'Joaquina gosta de João.'),
 (0.04313617836985864, 'Amanhã será um dia ensolarado.'),
 (0.04464102367697765, 'Maria, Pedro e João estudam na mesma escola.'),
 (0.045788086980364524, 'João gostava de estudar Ciência de Dados.'),
 (0.047535127376598096,
  'As ondas do mar estavam agitadas no último final de semana.'),
 (0.04826739208787742, 'O gato de João fugiu de casa na semana passada.'),
 (0.048889674494748805, 'Maria estuda no colégio mais caro da cidade.'),
 (0.05073106101214586, 'Pedro detesta Maria.'),
 (0.05095684430171058, 'O pôr do sol estava magnífico naquele dia.'),
 (0.05261732949249774, 'Eles voltaram muito tarde da festa de ontem.'),
 (0.056060921840691585, 'O cãozinho de Maria é muito bagunceiro.')]

### Vizinho mais próximo '<->' (No WHERE)

In [37]:
cursor.execute('SELECT frase_original FROM frases ORDER BY vetor_medio <-> %s;', (str(vetor_medio_teste.tolist()),))
cursor.fetchall()

[('Pedro gosta de comer chocolate.',),
 ('Jonas detesta chocolate amargo.',),
 ('Joaquina gosta de João.',),
 ('Amanhã será um dia ensolarado.',),
 ('Maria, Pedro e João estudam na mesma escola.',),
 ('João gostava de estudar Ciência de Dados.',),
 ('As ondas do mar estavam agitadas no último final de semana.',),
 ('O gato de João fugiu de casa na semana passada.',),
 ('Maria estuda no colégio mais caro da cidade.',),
 ('Pedro detesta Maria.',),
 ('O pôr do sol estava magnífico naquele dia.',),
 ('Eles voltaram muito tarde da festa de ontem.',),
 ('O cãozinho de Maria é muito bagunceiro.',)]

### Similaridade de coesseno '1 - (coluna_vetor <#> vetor_para_comparar)'

In [38]:
cursor.execute('SELECT 1 - (vetor_medio <=> %s) AS similaridade_cosseno, frase_original FROM frases ORDER BY similaridade_cosseno DESC;', (str(vetor_medio_teste.tolist()),))
cursor.fetchall()

[(0.7425666315285069, 'Pedro gosta de comer chocolate.'),
 (0.4231896731969361, 'Jonas detesta chocolate amargo.'),
 (0.3891526566604284, 'Joaquina gosta de João.'),
 (0.2454602554859613, 'Amanhã será um dia ensolarado.'),
 (0.11682969887498029, 'João gostava de estudar Ciência de Dados.'),
 (0.09552242662101251, 'Maria, Pedro e João estudam na mesma escola.'),
 (0.07929196358805435, 'Pedro detesta Maria.'),
 (0.006435425510865844, 'O gato de João fugiu de casa na semana passada.'),
 (0.0012604440712027376, 'Maria estuda no colégio mais caro da cidade.'),
 (-0.025241578625914807,
  'As ondas do mar estavam agitadas no último final de semana.'),
 (-0.028630789981419547, 'O cãozinho de Maria é muito bagunceiro.'),
 (-0.08609496757887203, 'O pôr do sol estava magnífico naquele dia.'),
 (-0.10649323757373264, 'Eles voltaram muito tarde da festa de ontem.')]

In [39]:
cursor.execute('SELECT (vetor_medio <=> %s) AS distancia_cosseno, frase_original FROM frases ORDER BY distancia_cosseno;', (str(vetor_medio_teste.tolist()),))
cursor.fetchall()

[(0.2574333684714931, 'Pedro gosta de comer chocolate.'),
 (0.5768103268030639, 'Jonas detesta chocolate amargo.'),
 (0.6108473433395716, 'Joaquina gosta de João.'),
 (0.7545397445140387, 'Amanhã será um dia ensolarado.'),
 (0.8831703011250197, 'João gostava de estudar Ciência de Dados.'),
 (0.9044775733789875, 'Maria, Pedro e João estudam na mesma escola.'),
 (0.9207080364119457, 'Pedro detesta Maria.'),
 (0.9935645744891342, 'O gato de João fugiu de casa na semana passada.'),
 (0.9987395559287973, 'Maria estuda no colégio mais caro da cidade.'),
 (1.0252415786259148,
  'As ondas do mar estavam agitadas no último final de semana.'),
 (1.0286307899814195, 'O cãozinho de Maria é muito bagunceiro.'),
 (1.086094967578872, 'O pôr do sol estava magnífico naquele dia.'),
 (1.1064932375737326, 'Eles voltaram muito tarde da festa de ontem.')]

In [40]:
cursor.execute('SELECT cosine_distance(vetor_medio, %s) AS distancia_cosseno, frase_original FROM frases ORDER BY distancia_cosseno;', (str(vetor_medio_teste.tolist()),))
cursor.fetchall()

[(0.2574333684714931, 'Pedro gosta de comer chocolate.'),
 (0.5768103268030639, 'Jonas detesta chocolate amargo.'),
 (0.6108473433395716, 'Joaquina gosta de João.'),
 (0.7545397445140387, 'Amanhã será um dia ensolarado.'),
 (0.8831703011250197, 'João gostava de estudar Ciência de Dados.'),
 (0.9044775733789875, 'Maria, Pedro e João estudam na mesma escola.'),
 (0.9207080364119457, 'Pedro detesta Maria.'),
 (0.9935645744891342, 'O gato de João fugiu de casa na semana passada.'),
 (0.9987395559287973, 'Maria estuda no colégio mais caro da cidade.'),
 (1.0252415786259148,
  'As ondas do mar estavam agitadas no último final de semana.'),
 (1.0286307899814195, 'O cãozinho de Maria é muito bagunceiro.'),
 (1.086094967578872, 'O pôr do sol estava magnífico naquele dia.'),
 (1.1064932375737326, 'Eles voltaram muito tarde da festa de ontem.')]

### Outras Funções
l1_distance(vetor, vetor) -> distância do taxi: normalmente usada para calcular distâncias entre dois poontos em um mapa de ruas.

l2_distance(vetor, vetor) -> distância euclidiana: geralmente utilizada em reconhecimento de padrões (identificação de objetos em imagens).

In [29]:
conn.close()